In [2]:
from operator import is_
import os
import sys
sys.path.append('../.')
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
import numpy as np

import clip

from Metrics import base_kmeans_model_evaluation, kmeans_with_init, cosine_kmeans_with_init
from networks import CustomCLIP, load_clip_to_cpu
from lr_scheduler import ConstantWarmupScheduler
from torch.utils.data import Dataset, DataLoader
import argparse

class custom_dataset(Dataset): 
    def __init__(self,  root_dir='../data/imagenet10', transform=None): 
        self.transform = transform
        data = np.load(os.path.join(root_dir,'data.npy'))
        self.targets = np.load(os.path.join(root_dir,'label.npy'))
        self.classes = list(set(self.targets))
        # b,h,w,c = data.shape
        # data = data.reshape((b, c, h, w))
        # self.data = data.transpose((0, 2, 3, 1))
        self.data = data
                
    def __getitem__(self, index):
        img, label = self.data[index], self.targets[index]
        img = Image.fromarray(img)
        img = self.transform(img) 
        return img,label
    def __len__(self):
        return len(self.data)
    
def dataset_select(dataset_name):
    if dataset_name == 'imagenet_dog':
        is_test_dataset = True
        fixed_dataset = custom_dataset(root_dir='../data/imagenet_dog', transform=preprocess)
        fixed_dataloader = DataLoader(
            dataset=fixed_dataset, 
            batch_size=batch_size,
            shuffle=False)
        dataset = custom_dataset(root_dir='../data/imagenet_dog', transform=preprocess)
        classes = ['cavachon','cavalier king charles spaniel','Basset Hound','Norwegian Elkhound',\
                  'Giant Schnauzer','golden retriever','brittany spaniel','clumber spaniel',\
                  'cocker spaniel','Belgian Shepherd Groenendael','Lancashire Heeler','rough collie','doberman pinscher','pug','chow chow']
    elif dataset_name == 'stanford40':
        is_test_dataset = True
        fixed_dataset = custom_dataset(root_dir='/database/daehyeon/Stanford40/JPEGImages', transform=preprocess)
        fixed_dataloader = DataLoader(
            dataset=fixed_dataset, 
            batch_size=batch_size,
            shuffle=False)
        dataset = custom_dataset(root_dir='/database/daehyeon/Stanford40/JPEGImages', transform=preprocess)
        classes = ['rowing a boat', 'blowing bubbles', 'phoning', 'playing guitar', 'fishing', 'pouring liquid', 'waving hands', 'writing on a book', 'holding an umbrella', 'riding a horse', 'washing dishes', 'fixing a bike', 'running', 'fixing a car', 'brushing teeth', 'climbing', 'cooking', 'jumping', 'looking through a microscope', 'feeding a horse', 'using a computer', 'reading', 'riding a bike', 'walking the dog', 'gardening', 'texting message', 'throwing frisby', 'cleaning the floor', 'drinking', 'pushing a cart', 'watching TV', 'cutting trees', 'looking through a telescope', 'cutting vegetables', 'shooting an arrow', 'smoking', 'applauding', 'writing on a board', 'taking photos', 'playing violin']

    elif dataset_name == 'mit':
        is_test_dataset = True
        fixed_dataset = custom_dataset(root_dir='/database/daehyeon/mit', transform=preprocess)
        fixed_dataloader = DataLoader(
            dataset=fixed_dataset, 
            batch_size=batch_size,
            shuffle=False)
        dataset = custom_dataset(root_dir='/database/daehyeon/mit', transform=preprocess)
        classes = ['poolinside','bedroom','inside_subway','operating_room','warehouse','clothingstore','toystore','waitingroom','lobby','casino','subway','closet','livingroom','laundromat','studiomusic','fastfood_restaurant','buffet','grocerystore','bakery','nursery','florist','airport_inside','artstudio','laboratorywet','videostore','kitchen','elevator','mall','deli','dentaloffice','greenhouse','bar','prisoncell','children_room','museum','gym','office','jewelleryshop','locker_room','hairsalon','restaurant','trainstation','pantry','dining_room','meeting_room','bookstore','tv_studio','bathroom','auditorium','bowling','shoeshop','restaurant_kitchen','hospitalroom','concert_hall','stairscase','corridor','garage','inside_bus','winecellar','computerroom','cloister','movietheater','kindergarden','library','classroom','gameroom','church_inside']
    return dataset,classes

In [54]:
backbone_name = "ViT-B/16"
device = torch.device('cuda') if torch.cuda.is_available() else "cpu"
clip_model, preprocess = load_clip_to_cpu(backbone_name)
n_ctx=16
dataset_name = 'imagenet_dog'

### dataset 

num_classes= len(testset.classes)
model = CustomCLIP(clip_model, num_classes, n_ctx=n_ctx)

for name, param in model.named_parameters():
    if "prompt_learner" not in name:
        param.requires_grad_(False)

model.to(device)
print('done')
#mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711)



Initializing class-specific contexts
Initial context: "age of personX X X X X X X X X X X X X X X X"
Number of context words (tokens): 16
done


In [3]:
batch_size =100
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=None)
testset.classes
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
#                                          shuffle=False)

# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                        download=True, transform=preprocess)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
#                                          shuffle=False)

Files already downloaded and verified


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [5]:
# batch_size = 50
# trainset = torchvision.datasets.STL10(
#     root='../data', split='train', download=True,
#     transform=preprocess)
# testset = torchvision.datasets.STL10(
#     root='../data', split='test', download=True,
#     transform=preprocess)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
#                                           shuffle=False)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
#                                           shuffle=False)
# concatset = torch.utils.data.ConcatDataset([trainset, testset])
# concatloader = torch.utils.data.DataLoader(concatset, batch_size=batch_size,
#                                          shuffle=False)



Files already downloaded and verified
Files already downloaded and verified


In [55]:
classes_list = ['a photo of a '+ c for c in testset.classes]
print(classes_list)
with torch.no_grad():
    classes_list = clip.tokenize(classes_list).to(device)
    centroid_candidate_text = clip_model.to(device).encode_text(classes_list)
    centroid_candidate_text = centroid_candidate_text / centroid_candidate_text.norm(dim=-1, keepdim=True)


TypeError: must be str, not numpy.int64

In [4]:
lr = 1e-5
criterion = nn.MSELoss()

dataset_name_list = ['cifar10']
for dataset_name in dataset_name_list:
    clip_backbone = "ViT-B/16"
    device = torch.device('cuda') if torch.cuda.is_available() else "cpu"
    backbone_name = clip_backbone
    clip_model, preprocess = load_clip_to_cpu(backbone_name)

#     testset, classes = dataset_select(dataset_name)
    batch_size =100
    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=preprocess)
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                           download=True, transform=preprocess)
    concatset = torch.utils.data.ConcatDataset([trainset, testset])
    concatloader = torch.utils.data.DataLoader(concatset, batch_size=batch_size,
                                             shuffle=False)
    classes = testset.classes
    
    model = CustomCLIP(clip_model, len(classes))
    model.to(device)

    classes_list = ['a photo of a '+ c for c in classes]
    print(classes_list)
    with torch.no_grad():
        classes_list = clip.tokenize(classes_list).to(device)
        centroid_candidate_text = clip_model.to(device).encode_text(classes_list)
        centroid_candidate_text = centroid_candidate_text / centroid_candidate_text.norm(dim=-1, keepdim=True)

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=5e-4,
        betas=(0.9, 0.999),
    )
    for i in range(1000):
        if i%50 == 0:
            print(i,'/',1000)
        prompts = model.prompt_learner().cuda()
        tokenized_prompts = model.tokenized_prompts
        for j in range(len(prompts)):
            text_feature = model.text_encoder(prompts[j], tokenized_prompts)
            text_feature = text_feature / \
                text_feature.norm(dim=-1, keepdim=True)
            if j == 0:
                text_features = text_feature.expand(1, -1, -1)
            else:
                text_feature = text_feature.expand(1, -1, -1)
                text_features = torch.cat((text_features, text_feature), dim=0)
        text_features = text_features.mean(dim=0)

        loss = criterion(text_features,centroid_candidate_text)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    model.eval()
    num_classes = 10
    with torch.no_grad():
        knn = cosine_kmeans_with_init
        new_label, acc, nmi,ari = knn(
            model, concatloader, num_classes, text_features)
    

Files already downloaded and verified
Files already downloaded and verified
Initializing class-specific contexts
Initial context: "age of personX X X X X X X X X X X X X X X X"
Number of context words (tokens): 16
['a photo of a airplane', 'a photo of a automobile', 'a photo of a bird', 'a photo of a cat', 'a photo of a deer', 'a photo of a dog', 'a photo of a frog', 'a photo of a horse', 'a photo of a ship', 'a photo of a truck']
0 / 1000
50 / 1000
100 / 1000
150 / 1000
200 / 1000
250 / 1000
300 / 1000
350 / 1000
400 / 1000
450 / 1000
500 / 1000
550 / 1000
600 / 1000
650 / 1000
700 / 1000
750 / 1000
800 / 1000
850 / 1000
900 / 1000
950 / 1000
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114


In [ ]:
print("acc,nmi,ari:",acc,nmi,ari)

In [59]:
lr = 1e-6
criterion = nn.MSELoss()

dataset_name_list = ['mit','stanford40','imagenet_dog']
for dataset_name in dataset_name_list:
    clip_backbone = "ViT-B/16"
    device = torch.device('cuda') if torch.cuda.is_available() else "cpu"
    backbone_name = clip_backbone
    clip_model, preprocess = load_clip_to_cpu(backbone_name)

    testset, classes = dataset_select(dataset_name)
    
    model = CustomCLIP(clip_model, len(classes))
    model.to(device)

    classes_list = ['a photo of a '+ c for c in classes]
    print(classes_list)
    with torch.no_grad():
        classes_list = clip.tokenize(classes_list).to(device)
        centroid_candidate_text = clip_model.to(device).encode_text(classes_list)
        centroid_candidate_text = centroid_candidate_text / centroid_candidate_text.norm(dim=-1, keepdim=True)

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=5e-4,
        betas=(0.9, 0.999),
    )
    for i in range(1000):
        if i%50 == 0:
            print(i,'/',1000)
        prompts = model.prompt_learner().cuda()
        tokenized_prompts = model.tokenized_prompts
        for j in range(len(prompts)):
            text_feature = model.text_encoder(prompts[j], tokenized_prompts)
            text_feature = text_feature / \
                text_feature.norm(dim=-1, keepdim=True)
            if j == 0:
                text_features = text_feature.expand(1, -1, -1)
            else:
                text_feature = text_feature.expand(1, -1, -1)
                text_features = torch.cat((text_features, text_feature), dim=0)
        text_features = text_features.mean(dim=0)

        loss = criterion(text_features,centroid_candidate_text)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        knn = cosine_kmeans_with_init
        new_label, acc, nmi,ari = knn(
            model, testloader, num_classes, text_features)
    

Initializing class-specific contexts
Initial context: "age of personX X X X X X X X X X X X X X X X"
Number of context words (tokens): 16
['a photo of a poolinside', 'a photo of a bedroom', 'a photo of a inside_subway', 'a photo of a operating_room', 'a photo of a warehouse', 'a photo of a clothingstore', 'a photo of a toystore', 'a photo of a waitingroom', 'a photo of a lobby', 'a photo of a casino', 'a photo of a subway', 'a photo of a closet', 'a photo of a livingroom', 'a photo of a laundromat', 'a photo of a studiomusic', 'a photo of a fastfood_restaurant', 'a photo of a buffet', 'a photo of a grocerystore', 'a photo of a bakery', 'a photo of a nursery', 'a photo of a florist', 'a photo of a airport_inside', 'a photo of a artstudio', 'a photo of a laboratorywet', 'a photo of a videostore', 'a photo of a kitchen', 'a photo of a elevator', 'a photo of a mall', 'a photo of a deli', 'a photo of a dentaloffice', 'a photo of a greenhouse', 'a photo of a bar', 'a photo of a prisoncell', 

In [42]:
classes = ['poolinside','bedroom','inside_subway','operating_room','warehouse','clothingstore','toystore','waitingroom','lobby','casino','subway','closet','livingroom','laundromat','studiomusic','fastfood_restaurant','buffet','grocerystore','bakery','nursery','florist','airport_inside','artstudio','laboratorywet','videostore','kitchen','elevator','mall','deli','dentaloffice','greenhouse','bar','prisoncell','children_room','museum','gym','office','jewelleryshop','locker_room','hairsalon','restaurant','trainstation','pantry','dining_room','meeting_room','bookstore','tv_studio','bathroom','auditorium','bowling','shoeshop','restaurant_kitchen','hospitalroom','concert_hall','stairscase','corridor','garage','inside_bus','winecellar','computerroom','cloister','movietheater','kindergarden','library','classroom','gameroom','church_inside']


In [52]:
list_ = os.listdir('/database/daehyeon/temporary/')
dict_ = {}
for item in list_:
    a = item.split('.')[0]
    b = a.split('_')[:-1]
    c = ' '.join(b)
    dict_[c] = 1
classes = list(dict_.keys())
print(len(classes))
print(classes)

40
['rowing a boat', 'blowing bubbles', 'phoning', 'playing guitar', 'fishing', 'pouring liquid', 'waving hands', 'writing on a book', 'holding an umbrella', 'riding a horse', 'washing dishes', 'fixing a bike', 'running', 'fixing a car', 'brushing teeth', 'climbing', 'cooking', 'jumping', 'looking through a microscope', 'feeding a horse', 'using a computer', 'reading', 'riding a bike', 'walking the dog', 'gardening', 'texting message', 'throwing frisby', 'cleaning the floor', 'drinking', 'pushing a cart', 'watching TV', 'cutting trees', 'looking through a telescope', 'cutting vegetables', 'shooting an arrow', 'smoking', 'applauding', 'writing on a board', 'taking photos', 'playing violin']
